# Fitting Thomson Scattering Spectrum

In [ ]:
%matplotlib inline
import ts_analysis as TsA
import matplotlib.pyplot as plt
import os
import numpy as np

# s0823_18 g23

### 1A: Load data and determine fibre positions on spectrometer CCD
First, you need to provide raw data in the form of a .asc file (plain ASCII, tab delimited) exported by Andor Solis for both the high power background and the shot. The background is used to calibrate the fibres. You can provide as many background files as you wish - the raw images are summed together. This is especially useful if you have pin scattering measurements for individual fibres, as the signal to noise ratio is far better than the high power background where the light scatters off particles in the vacuum chamber. To create a Thomson Scattering Analysis object, we use:

`ts=TsA.TS_Analysis('folder/shot_filename.asc',['folder/background_file1.asc','folder/background_file2.asc'])`

`ts` is an object that knows about the data in the files. `ts` starts by asking you where the fibre edges are. A graph appears on the screen with two sliders labelled `spacing` and `offset`. `spacing` is the number of pixels per fibres and `offset` is where the first fibre begins. Move the sliders so that the red dots lie in between each fibre in the minima. It is difficult to get the red dots perfect for every fibre, but this only amounts to an error of 1 pixel, so it is not significant.


In [ ]:
folder=r"example_data"
shot_fn='g023.asc'
bk_fn='gain3000slit120um.asc'
ts=TsA.TS_Analysis(folder, shot_fn,[bk_fn])
#You can write the correct spacing and offset here to remind yourself what to use when you rerun the code:
#spacing 55, offset 132

ts.find_fibre_edges()

### 1B: Specify fibre angles and split CCD into separate fibres
Once we have told `ts` where the fibre edges are, it can split the background and shot images into fibres and average over the pixels that correspond to each fibre:

`split_into_fibres(discard_rows=6, dark_counts = None, dark_x=300, intensity_x=60, fibres = 28):` 

* Discard rows is useful to get rid of low signal:noise ratio rows at the top and bottom of each fibre. You can discard the first and last n rows of each fibre to avoid averaging over this noise.
* dark_counts allows you to manually set the dark counts on the CCD. In this example, we find the dark counts in a small square in the top left where there is no signal.
* if you set dark_counts to None, you need to set dark_x, and the code will take the average dark counts in a strip [:,:x] and [:, -x:]. If you have no dark strips (for example, with the new spectrometer), use dark_counts instead.
* intensity_x is used to normalise the fibres, using the intensity in [mx-intensity_x:mx+intensity_x], where mx is the centre of the CCD. So if your signal is in a narrow region, like with IAW, use intensity_x=60 or so. For EPW the entire width of the CCD, so 250 or so.
* fibres: set the number of fibres you can see on the CCD. Usually 14 or 28.

`ts.zero_fibres(upper=1150, lower=850)` Trim the data to the edge of the active CCD region. Defaults are for IAW on the old spectrometer. This example file for EPW uses the entire CCD.

`ts.pair_fibres(a_angle,b_angle)` pair the background and shot fibres together, assign an angle to each fibre bundle. a_angle and b_angle are lists of angles which are N fibres long, eg. [45,45...] 

In [ ]:
dark_counts=int(ts.shot[:100,:100].mean())

ts.split_into_fibres(discard_rows=10, dark_counts=dark_counts, dark_x=None, intensity_x=250, fibres=14)

ts.zero_fibres(upper=1024, lower=0)
a_angle=[24]*7 #here we have fibres 1A through 7A at an angle of 24'
b_angle=[90]*7 #here we have fibres 1B through 7B at an angle of 24'
ts.pair_fibres(a_angle,b_angle)

## 2. Determine notch

We want to filter out the 'parasitic' or 'stray' light and the ion feature, which are close to the unshifted wavelength. To do this, we define a synthetic notch filter, which is applied to both the data and the model. You can test what size notch is useful below.

In [ ]:
f=ts.select_fibre(4,'A')

fig,ax=plt.subplots()
ax.plot(f.shot)

notch=np.ones_like(f.shot)
notch[440:550]=0 #edit the indices of this array to see the notch below
ax.plot(f.shot*notch)

## 3: Fit spectra
- Specify the fibre to fit using Fnum=(1-14) and Fset=('A' or 'B') (note quote marks)

- Leave the model as 'nLTE', no other models are currently implemented.

- Specify the plasma parameters in the form (Values, Independent?, Minimum value)

- If Independent is True, then the value is fixed and minimum has no meaning. If Independent is False, then the value is an initial guess for the fit and minimum is either a float, or simply ommitted to specify no given minimum.

- Leave A in brackets, eg 'A':  ((12), True). The code will find the relevant nLTE data for Z, T_e and decompose ZT_e into Z and T_e separately.

Once you've fitted the data, the code will remember the fit parameters it used, and instead of refitting you can simply plot or export the data using the next cells.

### Fibre 4A

In [ ]:
Fnum=4
Fset='A'

plasma_parameters={ 'model': 'electron',
                    'n_e': (3e17,False,1e16),#in cm^-3, must be specified
                    'T_e': (8.6, False,0.5),#In electron volts. 
                    'V_fe':(0, True),#m/s.
                    'stray':(0, True,0), #Fraction of signal made up of stray light. >0, <1.0!!
                    'amplitude': (3e3, False,0), #Fraction of signal made up of spectrum.
                    'offset': (0, False), #
                    'shift': (0, True)
                    }

'''This code fits and plots the result'''
f=ts.select_fibre(Fnum,Fset)
f.voigt_response(gamma=5e-11,sigma=6e-11)
f.symmetric_crop_around_l0()

notch=np.ones_like(f.shot)
notch[430:550]=0

f.fit_fibre(plasma_parameters, interpolation_scale=50, notch=notch)
f.calculate_alpha()
ts.pretty_plot(Fnum,Fset, tm=1.6, sr=100) #you can change the angstrom range plotted by altering 'sr=' here.
print(f.skw_res.fit_report()) #this prints the fitted values and the fit quality

ts.ax.set_ylim([0,5e3])#
ts.ax.set_xticks([-100,-50,0,50,100]);

### Fibre 4B

In [ ]:
Fnum=4
Fset='B'

plasma_parameters={ 'model': 'electron',
                    'n_e': (3e17,False,1e16),#in cm^-3, must be specified
                    'T_e': (5, False,0.5),#In electron volts. 
                    'V_fe':(0, True),#m/s.
                    'stray':(0, True,0), #Fraction of signal made up of stray light. >0, <1.0!!
                    'amplitude': (3e3, False,0), #Fraction of signal made up of spectrum.
                    'offset': (0, False), #
                    'shift': (0, True)
                    }

'''This code fits and plots the result'''
f=ts.select_fibre(Fnum,Fset)
f.voigt_response(gamma=5e-11,sigma=6e-11)
f.symmetric_crop_around_l0()

notch=np.ones_like(f.shot)
notch[430:550]=0

f.fit_fibre(plasma_parameters, interpolation_scale=50, notch=notch)
f.calculate_alpha()
ts.pretty_plot(Fnum,Fset, tm=1.6, sr=100) #you can change the angstrom range plotted by altering 'sr=' here.
print(f.skw_res.fit_report()) #this prints the fitted values and the fit quality

ts.ax.set_ylim([0,5e3])#
ts.ax.set_xticks([-100,-50,0,50,100]);

### 3A: Plot data
If you've already fitted a fibre and simply want to see the fit again without refitting, use the cell below. You can then save the figure using the next cell. You can change the range of the horizontal access by altering sr=, which is in Angstroms

In [ ]:
ts.pretty_plot(Fnum=4,Fset='A', tm=1.6, sr=6)
ts.ax.set_ylim([0,7e4])#
ts.ax.set_xticks([-100,-50,0,50,100]);

### 3B: Save plot
This saves whichever figure you have generated last, either by fitting (2) or using the plot data cell (3A) above. You can alter the filename

In [ ]:
ts.fig.savefig(ts.s_name+' fibre '+str(Fnum)+Fset+'.png')

### 4: Export data
Save out the fits so you can plot them in another plotting program. Specify the fibre number and the fibre set.

In [ ]:
for Fnum in [1,2,3,4,5,6,7]:
    f=ts.select_fibre(Fnum,'A')
    f.export_data('g23 '+str(Fnum)+'E')
    f=ts.select_fibre(Fnum,'B')
    f.export_data('g23 '+str(Fnum)+'F')

In [ ]:
import csv
def export_fit_parameters(fibres, fn):
    n_e=[]
    n_e_err=[]
    T_e=[]
    T_e_err=[]
    
    headings=('Fibre', 'n_e', 'n_e_err', 'T_e ', 'T_e_err')
    units=('-', ' cm^-3', ' cm^-3','eV','eV')
    with open(fn+'.dat', 'w',newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headings)
        writer.writerow(units)
        for i,fibre in enumerate(fibres):
            try:
                n_e=fibre.skw_res.params['n_e']
                n_e_err=n_e.stderr
                n_e=n_e.value
            except:
                n_e=fibre.params['n_e'] 
                n_e_err=0
            try:
                T_e=fibre.skw_res.params['T_e']
                T_e_err=T_e.stderr
                T_e=T_e.value
            except:
                T_e=fibre.params['T_e']
                T_e_err=0
            row=[i,n_e,n_e_err,T_e,T_e_err]
            writer.writerow(row)       

In [ ]:
export_fit_parameters(ts.fibres_a, fn='g23 E fibre fit parameters')
export_fit_parameters(ts.fibres_b, fn='g23 F fibre fit parameters')

### 5A: Plot all the A fibre data

In [ ]:
for i in range(1,8):
    ts.plot_data(Fnum=i, Fset='A', sr=50)

### 5B: Plot all the B fibre data

In [ ]:
for i in range(1,7):
    ts.plot_data(Fnum=i, Fset='B',sr=50)